## this is to run the model with the best regression and RMSE score for the model determined before 

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter


In [4]:
# Load the data
file_path = Path('happiness_joined.csv')
happiness_joined_df = pd.read_csv(file_path,encoding="ISO-8859-1")#,index_col=0)
# happiness_joined_df = df.loc[:, columns].copy()

# df.reset_index(inplace=True, drop=True)

happiness_joined_df.head()


,country_code,country_name,life_ladder,democracy_index,consumer_price_index,gender_ratio_males_per100_female,infant_mortality_per1000_births,life_expectancy,per_capita_gdp_dollars,population_density,safe_drinking_water_access_pct,seats_held_by_women_pct,unemployment_rate
0,8,Albania,4.995,58.9,119.1,103.7,8.0,78.4,5303,105.1,70.7,29.3,12.8
1,12,Algeria,4.745,40.1,151.4,102.1,21.2,76.6,3976,18.1,72.4,25.8,11.5
2,51,Armenia,5.488,55.4,129.2,88.8,10.8,74.9,4623,103.9,86.9,24.2,16.6
3,40,Austria,7.195,82.9,118.1,97.0,3.2,81.4,49701,108.7,98.9,37.2,4.8
4,31,Azerbaijan,5.173,27.5,156.9,99.7,20.8,72.8,4782,121.6,88.3,16.8,6.0


In [5]:
happiness_joined_df[['country_code','democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']] = happiness_joined_df[['country_code','democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']].replace({'\$': '', ',': ''}, regex=True).astype(float)
#
happiness_joined_df.head()
# # REF: https://pbpython.com/currency-cleanup.html

,country_code,country_name,life_ladder,democracy_index,consumer_price_index,gender_ratio_males_per100_female,infant_mortality_per1000_births,life_expectancy,per_capita_gdp_dollars,population_density,safe_drinking_water_access_pct,seats_held_by_women_pct,unemployment_rate
0,8.0,Albania,4.995,58.9,119.1,103.7,8.0,78.4,5303.0,105.1,70.7,29.3,12.8
1,12.0,Algeria,4.745,40.1,151.4,102.1,21.2,76.6,3976.0,18.1,72.4,25.8,11.5
2,51.0,Armenia,5.488,55.4,129.2,88.8,10.8,74.9,4623.0,103.9,86.9,24.2,16.6
3,40.0,Austria,7.195,82.9,118.1,97.0,3.2,81.4,49701.0,108.7,98.9,37.2,4.8
4,31.0,Azerbaijan,5.173,27.5,156.9,99.7,20.8,72.8,4782.0,121.6,88.3,16.8,6.0


In [6]:
happiness_joined_df.columns.values.tolist()

['country_code',
 'country_name',
 'life_ladder',
 'democracy_index',
 'consumer_price_index',
 'gender_ratio_males_per100_female',
 'infant_mortality_per1000_births',
 'life_expectancy',
 'per_capita_gdp_dollars',
 'population_density',
 'safe_drinking_water_access_pct',
 'seats_held_by_women_pct',
 'unemployment_rate']

In [7]:
print(happiness_joined_df.shape)

(91, 13)


## Modeling -- Random Forest regression using all 8 indep variables except "country_name", "country_code",  "life_expectancy" & "consumer_price_index"

In [8]:
# select features 
target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name"])#, "life_expectancy", "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]
import statsmodels.api as sm
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

# either 5 variables or 10, but with splits

                            OLS Regression Results                            
Dep. Variable:            life_ladder   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     23.60
Date:                Mon, 31 Jan 2022   Prob (F-statistic):           5.95e-20
Time:                        11:09:31   Log-Likelihood:                -71.284
No. Observations:                  91   AIC:                             164.6
Df Residuals:                      80   BIC:                             192.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [24]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.92)

In [25]:
from scipy import stats

from sklearn.linear_model import BayesianRidge, LinearRegression


# #############################################################################
# Fit the Bayesian Ridge Regression and an OLS for comparison
clf = BayesianRidge(compute_score=True)
clf.fit(X_train, y_train)
from sklearn.metrics import mean_squared_error
y_pred = clf.predict(X_train)
mse = mean_squared_error(y_train, y_pred)



from sklearn.preprocessing import StandardScaler
# 3) Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# 4) Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_train)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(83, 10)
(8, 10)
(83,)
(8,)


In [26]:
print(np.mean(X_train_scaled[:,0]))
print(np.mean(X_test_scaled[:,0]))
print(np.std(X_train_scaled[:,0]))
print(np.std(X_test_scaled[:,0]))

-7.490661370964911e-17
-7.490661370964911e-17
1.0000000000000002
1.0000000000000002


In [28]:
# model evaluation
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b
from sklearn.metrics import r2_score, mean_squared_error
score = r2_score(y_train, y_pred)
print("R2score:", score)
rmse = mean_squared_error(y_train, y_pred, squared=False)
print('root mean squared error: ', rmse)
# ref: https://datagy.io/python-sklearn-linear-regression/

R2score: 0.7176865692306966
root mean squared error:  0.5670411265493204


In [32]:
# Make predictions using the test data
y_pred = clf.predict(X_test)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,5.433118,5.653
1,5.658461,5.809
2,4.664614,5.821
3,6.107737,6.064
4,5.691147,6.016


In [33]:
# The slope
# The y-intercept
print(clf.coef_)
print(clf.intercept_)

[ 4.71082854e-03 -2.59142127e-03  3.87615779e-03 -5.82112304e-03
  1.45600740e-03  2.08544234e-05 -7.82269917e-05  6.51740339e-03
  5.06006659e-03 -2.52004845e-03]
4.520800536011755


In [36]:
# summary_df that shows the line equation of the model: 
# initialize list of lists
data = [['Bayessian Linear Regression', score, rmse, clf.intercept_, clf.coef_[0], clf.coef_[1], clf.coef_[2], clf.coef_[3], clf.coef_[4], clf.coef_[5], clf.coef_[6], clf.coef_[7], clf.coef_[8], clf.coef_[9]]]
 
# Create the pandas DataFrame
summary_df = pd.DataFrame(data, columns = ['Name of model:', 'R-squared_score', 'RMSE_Score', 'intercept', 'coeff1', 'coeff2', 'coeff3' ,'coeff4', 'coeff5','coeff6', 'coeff7', 'coeff8', 'coeff9', 'coeff10'])

summary_df.round(5)

,Name of model:,R-squared_score,RMSE_Score,intercept,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10
0,Bayessian Linear Regression,0.71769,0.56704,4.5208,0.00471,-0.00259,0.00388,-0.00582,0.00146,0.00002,-0.00008,0.00652,0.00506,-0.00252


In [ ]:
# rerun the significant value >0.05 value for the indep cvariables
# seperate with subheadings for each variables 

In [ ]:
# select features 
#X = happiness_joined_df[['democracy_index','consumer_price_index','gender_ratio_males_per100_female','infant_mortality_per1000_births','life_expectancy','per_capita_gdp_dollars','population_density','safe_drinking_water_access_pct','seats_held_by_women_pct','unemployment_rate']]
#y = happiness_joined_df["life_ladder"]
# ref: https://www.pluralsight.com/guides/importing-and-splitting-data-into-dependent-and-independent-features-for-ml
# ref: https://towardsdatascience.com/multiple-regression-as-a-machine-learning-algorithm-a98a6b9f307b


target = "life_ladder"
features = happiness_joined_df.columns.drop([target,"country_code","country_name", "life_expectancy", "consumer_price_index", "democracy_index", "infant_mortality_per1000_births","gender_ratio_males_per100_female", "population_density"])#,"unemployment_rate"])
X = happiness_joined_df[features]
y = happiness_joined_df[target]